In [4]:
import pandas as pd
from os import walk

In [5]:
destination = r'/Users/clausholm/Documents/Python/Master_thesis/Data/German_radio_cleaned/DailyPlays/Random10OverAndUnder'

f = []
for (dirpath, dirnames, filenames) in walk(destination):
    f.extend(filenames)
    break
print(f)

['C_Arma_-_Yapma_-_Radio_Edit_5_over.csv', 'Soleima - Low Life_1_over.csv', 'C_Arma_-_Yapma_-_Radio_Edit_4_over.csv', 'Moses Sumney - Title Of This Song_3_over.csv', 'Moses Sumney - Title Of This Song_2_over.csv', 'Glasperlenspiel_-_Geiles_Leben_2_over.csv', 'Robyn - Honey_2_under.csv', 'Glasperlenspiel_-_Geiles_Leben_3_over.csv', 'Ariana_Grande_-_7_rings_5_under.csv', 'Johannes_Oerding_-_An_guten_Tagen_2_under.csv', 'Rosalía - MALAMENTE (Cap.1: Augurio)_4_under.csv', "Dynoro_&_Gigi_D'agostino_-_In_My_Mind_2_under.csv", 'LEA_&_Cyril_-_Immer_wenn_wir_uns_sehn_4_over.csv', 'LEA_&_Cyril_-_Immer_wenn_wir_uns_sehn_5_over.csv', 'Gaullin_-_Moonlight_1_over.csv', 'David_Guetta_ft._Bebe_Rexha_J_Balvin_-_Say_My_Name_4_over.csv', 'David_Guetta_ft._Bebe_Rexha_J_Balvin_-_Say_My_Name_5_over.csv', 'Dominic_Fike_-_3_Nights_5_under.csv', 'Larsito_&_Mandy_Capristo_-_Si_es_amor_3_over.csv', 'Larsito_&_Mandy_Capristo_-_Si_es_amor_2_over.csv', 'Johannes_Oerding_-_An_guten_Tagen_2_over.csv', 'Johannes_Oerd

In [8]:
for i in f:
    #try:
        
    # Import .csv-file
    df = pd.read_csv (destination+"/"+i,sep=';')

    artist = df.Artist[0]
    title = df.Title[0]

    print(artist)

    try:
        df = df.drop(columns=['Column0', 'Column1','Column2','Column3','Column4','Column5','Column6','Column7'])

    except:
        print("no ekstra columns")

    #df = df.drop(columns=['Artist','ISRC','Duration','Title','City','Country'])

    pivotDf = df

    df['DateTime'] = df['DateTime'].str[:10]
    df['Play'] = 1

    df['DateTime'] = df['DateTime'].astype('datetime64[ns]')

    pivotDf = pivotDf.pivot_table(index='DateTime', columns='Channel',aggfunc=sum)

    pivotDf.fillna(0,inplace=True)
    pivotDf = pivotDf.astype(int)

    pivotDf.columns = pivotDf.columns.droplevel(0) #remove amount
    pivotDf.columns.name = None               #remove categories
    pivotDf = pivotDf.reset_index()                #index to columns

    pivotDf['DateTime'] = pivotDf['DateTime'].astype('datetime64[ns]')

    minVal = pivotDf['DateTime'].min()

    d = {'Date': [minVal],'TotalPlays':[""]}
    emptyDf = pd.DataFrame(data=d)
    emptyDf['Date'] = emptyDf['Date'].astype('datetime64[ns]')

    while len(emptyDf) < 180:
        maxVal = emptyDf['Date'].max()+pd.DateOffset(days=1)
        row = pd.Series([maxVal,""], index=['Date', 'TotalPlays'])
        emptyDf = emptyDf.append(row, ignore_index=True)

    emptyDf.reset_index()
    emptyDf = emptyDf.set_index('Date')

    emptyDf['TotalPlays'] =df['DateTime'].value_counts()
    emptyDf.fillna(0,inplace=True)
    emptyDf = emptyDf.astype(int)

    pivotDf = pivotDf.set_index('DateTime')

    conDf = pd.concat([emptyDf,pivotDf],axis=1,sort=False)

    conDf.fillna(0,inplace=True)
    conDf = conDf.astype(int)

    conDf = conDf.reset_index()

    conDf.rename(columns={"index": "Date"})

    conDf.insert(loc=1, column='Artist', value=artist)
    conDf.insert(loc=2, column='Title', value=title)

    conDf.to_excel(destination + "/Cleaned/" + artist + " - " + title + ".xlsx")

    #except:
    #   print("File not found")

C Arma
no ekstra columns


KeyError: 'DateTime'